In [23]:
import cv2
import numpy as np
from keras import models
from keras import layers

## Pré-processamento

In [7]:
char_to_int = {"0" : 0, "1" : 1, "2" : 2, "3" : 3, "4" : 4, "5" : 5, "6" : 6, "7" : 7, "8" : 8, "9" : 9, 
               "A" : 10, "B" : 11, "C" : 12, "D" : 13, "E" : 14, "F" : 15, "G" : 16, "H" : 17, "I" : 18, 
               "J" : 19, "K" : 20, "L" : 21, "M" : 22, "N" : 23, "O" : 24, "P" : 25, "Q" : 26, "R" : 27, 
               "S" : 28, "T" : 29, "U" : 30, "V" : 31, "X" : 32, "W" : 33, "Y" : 34, "Z" : 35, "?": 36}

In [31]:
def preProcessamento():

    x_treino, x_validacao, x_teste = [], [], []
    y_treino, y_validacao, y_teste = [], [], []
    
    for i in range(1, 10001):
        
        arq = open("labels10k/" + f"{i:06}" + ".txt", "r")
        line = arq.readline()
        arq.close()
        chars = list(line)
        
        if i <= 8000:
            caminho_img = "treinamento/" + f"{i:06}" + ".jpg" 
        elif 8000 < i <= 9000:
            caminho_img = "validacao/" + f"{i:06}" + ".jpg"
        else:
            caminho_img = "teste/" + f"{i:06}" + ".jpg"
    
        img = cv2.imread(caminho_img)
        img_cinza = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_bin = cv2.threshold(img_cinza, 16, 255, cv2.THRESH_BINARY)[1]
        img_median = cv2.medianBlur(img_bin, 5)

        k = 0
        for j in range(10,img.shape[1],30):

            aux = img_median[:,j:j+30]
            aux = cv2.resize(aux,(40,40))
            aux = cv2.threshold(aux, 127, 255, cv2.THRESH_BINARY)[1]
            aux = aux/255
            aux = aux.flatten()
            
            if i <= 8000:
                x_treino.append(aux)
                y_treino.append(char_to_int[chars[k]])
                k+=1
            elif 8000 < i <= 9000:
                x_validacao.append(aux)
                y_validacao.append(char_to_int[chars[k]])
                k+=1
            else:
                x_teste.append(aux)
                y_teste.append(char_to_int[chars[k]])
                k+=1
            
    return [(np.array(x_treino), np.array(y_treino)),
            (np.array(x_validacao), np.array(y_validacao)),
            (np.array(x_teste), np.array(y_teste))]

In [32]:
resultados = preProcessamento()

In [45]:
resultados[2][0][1].shape

(1600,)

In [52]:
num = 173
print(str(resultados[0][1][num]))
cv2.imshow("teste", (resultados[0][0][num]*255).reshape((40,40)))
cv2.waitKey()
cv2.destroyAllWindows()

16


## Classificação com redes neurais densas

In [73]:
network = models.Sequential()
network.add(layers.Dense(850, activation='relu', input_shape=(1600,)))
network.add(layers.Dense(850, activation='relu'))
network.add(layers.Dense(37, activation='relu'))

network.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [74]:
print(network.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 850)               1360850   
                                                                 
 dense_25 (Dense)            (None, 850)               723350    
                                                                 
 dense_26 (Dense)            (None, 37)                31487     
                                                                 
Total params: 2,115,687
Trainable params: 2,115,687
Non-trainable params: 0
_________________________________________________________________
None


In [64]:
history = network.fit(resultados[0][0], resultados[0][1], epochs=20, batch_size=600, validation_data=(resultados[1][0], resultados[1][1]))

Epoch 1/20
94/94 [==============================] - 7s 69ms/step - loss: -207.3500 - accuracy: 0.0291 - val_loss: -209.1311 - val_accuracy: 0.0283
Epoch 2/20
94/94 [==============================] - 6s 66ms/step - loss: -211.1061 - accuracy: 0.0293 - val_loss: -209.1311 - val_accuracy: 0.0283
Epoch 3/20
94/94 [==============================] - 7s 70ms/step - loss: -211.1061 - accuracy: 0.0293 - val_loss: -209.1311 - val_accuracy: 0.0283
Epoch 4/20
94/94 [==============================] - 6s 67ms/step - loss: -211.1060 - accuracy: 0.0293 - val_loss: -209.1311 - val_accuracy: 0.0283
Epoch 5/20
94/94 [==============================] - 7s 71ms/step - loss: -211.1061 - accuracy: 0.0293 - val_loss: -209.1311 - val_accuracy: 0.0283
Epoch 6/20
94/94 [==============================] - 7s 73ms/step - loss: -211.1060 - accuracy: 0.0293 - val_loss: -209.1311 - val_accuracy: 0.0283
Epoch 7/20
94/94 [==============================] - 7s 71ms/step - loss: -211.1060 - accuracy: 0.0293 - val_loss: -209

In [67]:
network.predict([resultados[2][0][0]])

ValueError: in user code:

    File "C:\Users\Arthur\anaconda3\lib\site-packages\keras\engine\training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Arthur\anaconda3\lib\site-packages\keras\engine\training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Arthur\anaconda3\lib\site-packages\keras\engine\training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Arthur\anaconda3\lib\site-packages\keras\engine\training.py", line 1791, in predict_step
        return self(x, training=False)
    File "C:\Users\Arthur\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Arthur\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 228, in assert_input_compatibility
        raise ValueError(f'Input {input_

In [72]:
resultados[2][0][0].shape

(1600,)

## Classificação com redes neurais convolucionais